In [ ]:
import os
import warnings
import anndata
import scanpy as sc
import numpy as np
import pandas as pd
from scBiMapping import scBiMapping_annotation

In [ ]:
adata_ref_raw = anndata.read_h5ad('Macaque.sn.h5ad') # reference data
sc.pp.normalize_total(adata_ref_raw, target_sum=1e4)
sc.pp.log1p(adata_ref_raw)

In [ ]:
 
dir_path = 'STDS0000244/' # Path of query slices
files = os.listdir(dir_path)
for file in files:
    adata_query = anndata.read(f'{dir_path}{file}')
    sc.pp.log1p(adata_query)

    intersection_feature = list(set(adata_ref_raw.var_names) & set(adata_query.var_names))
    adata_ref = adata_ref_raw[:,intersection_feature].copy()
    adata_query = adata_query[:,intersection_feature]
 
    # parameter setting 
    n_embedding = 30; K = 30; K_majority = 30; knnMethod = 'HNSW'; # one can also use: knnMethod = 'NNDescent'
    adata_ref.obs['cell_annotation'] = adata_ref.obs['cluster']  # 15 cell types
    # runing
    celltype_predicted = scBiMapping_annotation(adata_ref, adata_query,normalization=normalization,n_embedding = n_embedding,K = K,K_majority = K_majority,knnMethod = knnMethod)
    adata_query.obs['celltype_scBiMapping'] = pd.Categorical(celltype_predicted)
    
    adata_query.obs.to_csv(f"scBiMapping_result/scBiMapping_{file.split('_')[2]}.csv") 